In [7]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Documents/UCL/3x2_analytical`
   Resolving package versions...
   Installed LERC_jll ───────────────────── v3.0.0+1
   Installed Opus_jll ───────────────────── v1.3.2+0
   Installed x265_jll ───────────────────── v3.5.0+0
   Installed libfdk_aac_jll ─────────────── v2.0.2+0
   Installed GR_jll ─────────────────────── v0.73.5+0
   Installed JpegTurbo_jll ──────────────── v3.0.3+0
   Installed LoggingExtras ──────────────── v1.0.3
   Installed Libmount_jll ───────────────── v2.40.0+0
   Installed Xorg_xkbcomp_jll ───────────── v1.4.6+0
   Installed Measures ───────────────────── v0.3.2
   Installed ConcurrentUtilities ────────── v2.4.1
   Installed RelocatableFolders ─────────── v1.0.1
   Installed Unitful ────────────────────── v1.19.0
   Installed Contour ────────────────────── v0.6.3
   Installed Xorg_xcb_util_wm_jll ───────── v0.4.1+1
   Installed Xorg_xcb_util_image_jll ────── v0.4.0+1
   Installed RecipesPipeline ────────────── v0.6.12
   Installed Grisu 

In [8]:
using LinearAlgebra
using LimberJack
using CSV
using YAML
using JLD2
using PythonCall
using DataFrames
using NPZ
using Plots
sacc = pyimport("sacc");

[ Info: Precompiling IJuliaExt [2f4121a4-3b3a-5ce6-9c5e-1f2673ce168a]


In [9]:
sacc_path = "../../PhD/LimberJack.jl/data/FD/cls_FD_covG.fits"
yaml_path_wlwl = "data/DESY1/wlwl.yml"
sacc_wlwl = sacc.Sacc().load_fits(sacc_path)
yaml_wlwl = YAML.load_file(yaml_path_wlwl)
meta_wlwl, files_wlwl = make_data(sacc_wlwl, yaml_wlwl)

DESwl__0 DESwl__0 24
DESwl__0 DESwl__1 24
DESwl__0 DESwl__2 24
DESwl__0 DESwl__3 24
DESwl__1 DESwl__1 24
DESwl__1 DESwl__2 24
DESwl__1 DESwl__3 24
DESwl__2 DESwl__2 24
DESwl__2 DESwl__3 24
DESwl__3 DESwl__3 24


(LimberJack.Instructions(["DESwl__0", "DESwl__1", "DESwl__2", "DESwl__3"], Any[["DESwl__0", "DESwl__0"], ["DESwl__0", "DESwl__1"], ["DESwl__0", "DESwl__2"], ["DESwl__0", "DESwl__3"], ["DESwl__1", "DESwl__1"], ["DESwl__1", "DESwl__2"], ["DESwl__1", "DESwl__3"], ["DESwl__2", "DESwl__2"], ["DESwl__2", "DESwl__3"], ["DESwl__3", "DESwl__3"]], ["galaxy_shear", "galaxy_shear", "galaxy_shear", "galaxy_shear"], [0, 24, 48, 72, 96, 120, 144, 168, 192, 216, 240], [2.4659965589802277e-9, 1.937338987992683e-9, 6.992765360481148e-10, -4.9611713239579213e-11, 7.913395793306234e-10, 2.025477514463439e-10, 2.937251108461027e-10, 2.5800941836058233e-10, 3.588061352802992e-10, 6.231168353367879e-10  …  4.737458899576856e-10, 8.27594560926968e-10, 5.318332277673948e-10, 3.860067333751191e-10, 4.674884118894536e-10, 3.737344096265107e-10, 8.676072775051082e-11, 2.8609099937088954e-10, 2.0381853638647756e-10, 1.2493271614464064e-10], [1.470692645100043e-18 -9.214475828663469e-20 … -8.136848663040252e-24 -7.

In [10]:
sacc_path = "../../PhD/LimberJack.jl/data/FD/cls_FD_covG.fits"
yaml_path_gcgc = "data/DESY1/gcgc.yml"
sacc_gcgc = sacc.Sacc().load_fits(sacc_path)
yaml_gcgc = YAML.load_file(yaml_path_gcgc)
meta_gcgc, files_gcgc = make_data(sacc_gcgc, yaml_gcgc)

DESgc__0 DESgc__0 5
DESgc__1 DESgc__1 8
DESgc__2 DESgc__2 10
DESgc__3 DESgc__3 11
DESgc__4 DESgc__4 13


(LimberJack.Instructions(["DESgc__0", "DESgc__1", "DESgc__2", "DESgc__3", "DESgc__4"], Any[["DESgc__0", "DESgc__0"], ["DESgc__1", "DESgc__1"], ["DESgc__2", "DESgc__2"], ["DESgc__3", "DESgc__3"], ["DESgc__4", "DESgc__4"]], ["galaxy_density", "galaxy_density", "galaxy_density", "galaxy_density", "galaxy_density"], [0, 5, 13, 23, 34, 47], [0.00019024248750512107, 7.391276663994371e-5, 4.261252890607883e-5, 2.22965965139283e-5, 2.176880112868429e-5, 8.240734039668606e-5, 8.421314587773898e-5, 2.6585137640401017e-5, 2.6033426315289483e-5, 1.6202481595102046e-5  …  1.720793463317388e-5, 1.876041365219758e-5, 1.1858834397319835e-5, 6.023987322753388e-6, 3.4894299563905405e-6, 7.420152296134288e-6, 4.729410691534381e-6, 4.317808418665961e-6, 4.736098712055406e-6, 2.545509196409628e-6], [3.503459378925734e-9 -2.6907237994012596e-10 … -1.8766243449428733e-32 -1.9859652134651026e-32; -2.6907237994012596e-10 2.4651642374142706e-10 … -4.810041560235908e-31 -4.501051455667023e-31; … ; -1.87662434494

In [11]:
sacc_path = "../../PhD/LimberJack.jl/data/FD/cls_FD_covG.fits"
yaml_path_3x2 = "data/DESY1/gcgc_gcwl_wlwl.yml"
sacc_3x2 = sacc.Sacc().load_fits(sacc_path)
yaml_3x2 = YAML.load_file(yaml_path_3x2) 
meta_3x2, files_3x2 = make_data(sacc_3x2, yaml_3x2)

DESgc__0 DESgc__0 5
DESgc__1 DESgc__1 8
DESgc__2 DESgc__2 10
DESgc__3 DESgc__3 11
DESgc__4 DESgc__4 13
DESgc__0 DESwl__0 5
DESgc__0 DESwl__1 5
DESgc__0 DESwl__2 5
DESgc__0 DESwl__3 5
DESgc__1 DESwl__0 8
DESgc__1 DESwl__1 8
DESgc__1 DESwl__2 8
DESgc__1 DESwl__3 8
DESgc__2 DESwl__0 10
DESgc__2 DESwl__1 10
DESgc__2 DESwl__2 10
DESgc__2 DESwl__3 10
DESgc__3 DESwl__0 11
DESgc__3 DESwl__1 11
DESgc__3 DESwl__2 11
DESgc__3 DESwl__3 11
DESgc__4 DESwl__0 13
DESgc__4 DESwl__1 13
DESgc__4 DESwl__2 13
DESgc__4 DESwl__3 13
DESwl__0 DESwl__0 24
DESwl__0 DESwl__1 24
DESwl__0 DESwl__2 24
DESwl__0 DESwl__3 24
DESwl__1 DESwl__1 24
DESwl__1 DESwl__2 24
DESwl__1 DESwl__3 24
DESwl__2 DESwl__2 24
DESwl__2 DESwl__3 24
DESwl__3 DESwl__3 24


(LimberJack.Instructions(["DESgc__0", "DESgc__1", "DESgc__2", "DESgc__3", "DESgc__4", "DESwl__0", "DESwl__1", "DESwl__2", "DESwl__3"], Any[["DESgc__0", "DESgc__0"], ["DESgc__1", "DESgc__1"], ["DESgc__2", "DESgc__2"], ["DESgc__3", "DESgc__3"], ["DESgc__4", "DESgc__4"], ["DESgc__0", "DESwl__0"], ["DESgc__0", "DESwl__1"], ["DESgc__0", "DESwl__2"], ["DESgc__0", "DESwl__3"], ["DESgc__1", "DESwl__0"]  …  ["DESwl__0", "DESwl__0"], ["DESwl__0", "DESwl__1"], ["DESwl__0", "DESwl__2"], ["DESwl__0", "DESwl__3"], ["DESwl__1", "DESwl__1"], ["DESwl__1", "DESwl__2"], ["DESwl__1", "DESwl__3"], ["DESwl__2", "DESwl__2"], ["DESwl__2", "DESwl__3"], ["DESwl__3", "DESwl__3"]], ["galaxy_density", "galaxy_density", "galaxy_density", "galaxy_density", "galaxy_density", "galaxy_shear", "galaxy_shear", "galaxy_shear", "galaxy_shear"], [0, 5, 13, 23, 34, 47, 52, 57, 62, 67  …  259, 283, 307, 331, 355, 379, 403, 427, 451, 475], [0.00019024248750512107, 7.391276663994371e-5, 4.261252890607883e-5, 2.22965965139283e-5

### Data

In [ ]:
CSV.write("data/DESY1/data_wlwl.csv",
    Dict{String}{Vector}(
        "data" => meta_wlwl.data,
        "err" => sqrt.(diag(meta_wlwl.cov))))

CSV.write("data/DESY1/data_gcgc.csv",
    Dict{String}{Vector}(
        "data" => meta_gcgc.data,
        "err" => sqrt.(diag(meta_gcgc.cov))))
    
CSV.write("data/DESY1/data_3x2.csv",
    Dict{String}{Vector}(
        "data" => meta_3x2.data,
        "err" => sqrt.(diag(meta_3x2.cov))))

In [ ]:
delete!(files_wlwl, "nz_DESwl__0")
delete!(files_wlwl, "nz_DESwl__1")
delete!(files_wlwl, "nz_DESwl__2")
delete!(files_wlwl, "nz_DESwl__3")

delete!(files_gcgc, "nz_DESgc__0")
delete!(files_gcgc, "nz_DESgc__1")
delete!(files_gcgc, "nz_DESgc__2")
delete!(files_gcgc, "nz_DESgc__3")
delete!(files_gcgc, "nz_DESgc__4")

delete!(files_3x2, "nz_DESwl__0")
delete!(files_3x2, "nz_DESwl__1")
delete!(files_3x2, "nz_DESwl__2")
delete!(files_3x2, "nz_DESwl__3")
delete!(files_3x2, "nz_DESgc__0")
delete!(files_3x2, "nz_DESgc__1")
delete!(files_3x2, "nz_DESgc__2")
delete!(files_3x2, "nz_DESgc__3")
delete!(files_3x2, "nz_DESgc__4")

In [ ]:
npzwrite("data/DESY1/files_wlwl.npz", files_wlwl)
npzwrite("data/DESY1/files_gcgc.npz", files_gcgc)
npzwrite("data/DESY1/files_3x2.npz", files_3x2)

### Make Data

In [12]:
function model_wlwl(x; meta=meta_wlwl, files=files_wlwl)

    nuisances = Dict(
        "DESgc__0_b" => 1.484,
        "DESgc__1_b" => 1.805,
        "DESgc__2_b" => 1.776,
        "DESgc__3_b" => 2.168,
        "DESgc__4_b" => 2.23,
        "DESwl__0_m" => 0.018,
        "DESwl__1_m" => 0.014,
        "DESwl__2_m" => 0.01,
        "DESwl__3_m" => 0.004,
        "A_IA" => 0.294,
        "alpha_IA" => 0.378)
    
    cosmology = Cosmology(Ωm=x[1], Ωb=x[2], h=x[3], σ8=x[4], ns=x[5],
        tk_mode=:EisHu,
        pk_mode=:Halofit)

    theory = Theory(cosmology, meta, files; Nuisances=nuisances)
    d = theory .- meta.data
    xi2 = d'*meta.inv_cov*d
    return theory, xi2
end

function model_gcgc(x; meta=meta_gcgc, files=files_gcgc)

    nuisances = Dict(
        "DESgc__0_b" => 1.484,
        "DESgc__1_b" => 1.805,
        "DESgc__2_b" => 1.776,
        "DESgc__3_b" => 2.168,
        "DESgc__4_b" => 2.23,
        "DESwl__0_m" => 0.018,
        "DESwl__1_m" => 0.014,
        "DESwl__2_m" => 0.01,
        "DESwl__3_m" => 0.004,
        "A_IA" => 0.294,
        "alpha_IA" => 0.378)

    cosmology = Cosmology(Ωm=x[1],  Ωb=x[2], h=x[3], σ8=x[4], ns=x[5],
        tk_mode=:EisHu,
        pk_mode=:Halofit)

    theory = Theory(cosmology, meta, files; Nuisances=nuisances)
    d = theory .- meta.data
    xi2 = d'*meta.inv_cov*d
    return theory, xi2
end

function model_3x2(x; meta=meta_3x2, files=files_3x2)

    nuisances = Dict(
        "DESgc__0_b" => 1.484,
        "DESgc__1_b" => 1.805,
        "DESgc__2_b" => 1.776,
        "DESgc__3_b" => 2.168,
        "DESgc__4_b" => 2.23,
        "DESwl__0_m" => 0.018,
        "DESwl__1_m" => 0.014,
        "DESwl__2_m" => 0.01,
        "DESwl__3_m" => 0.004,
        "A_IA" => 0.294,
        "alpha_IA" => 0.378)

    cosmology = Cosmology(Ωm=x[1],  Ωb=x[2], h=x[3], σ8=x[4], ns=x[5],
        tk_mode=:EisHu,
        pk_mode=:Halofit)

    theory = Theory(cosmology, meta, files; Nuisances=nuisances)
    d = theory .- meta.data
    xi2 = d'*meta.inv_cov*d
    return theory, xi2
end

model_3x2 (generic function with 1 method)

In [13]:
path_dzs_wlwl_nomarg = "chains/nomarg/DES_wlwl_nomarg_TAP_0.65_init_ϵ_0.03/"
path_dzs_gcgc_nomarg = "chains/nomarg/DES_gcgc_nomarg_TAP_0.65_init_ϵ_0.03/"
path_dzs_3x2_nomarg = "chains/nomarg/DES_3x2_nomarg_TAP_0.65_init_ϵ_0.03/"

path_dzs_wlwl_num = "chains/numerical/DES_wlwl_dz_num_TAP_0.65_init_ϵ_0.03/"
path_dzs_gcgc_num = "chains/numerical/DES_gcgc_dz_num_TAP_0.65_init_ϵ_0.03/"
path_dzs_3x2_num = "chains/numerical/DES_3x2_dz_num_TAP_0.65_init_ϵ_0.03/"

path_dzs_wlwl_ana = "chains/analytical/DES_wlwl_dz_ana_TAP_0.65_init_ϵ_0.03/"
path_dzs_gcgc_ana = "chains/analytical/DES_gcgc_dz_ana_TAP_0.65_init_ϵ_0.03/"
path_dzs_3x2_ana = "chains/analytical/DES_3x2_dz_ana_TAP_0.65_init_ϵ_0.03/"

chain_wlwl_nomarg = CSV.read(path_dzs_wlwl_nomarg*"chain_1.csv", DataFrame);
chain_gcgc_nomarg = CSV.read(path_dzs_gcgc_nomarg*"chain_1.csv", DataFrame);
chain_3x2_nomarg = CSV.read(path_dzs_3x2_nomarg*"chain_1.csv", DataFrame);

chain_wlwl_num = CSV.read(path_dzs_wlwl_num*"chain_1.csv", DataFrame);
chain_gcgc_num = CSV.read(path_dzs_gcgc_num*"chain_1.csv", DataFrame);
chain_3x2_num = CSV.read(path_dzs_3x2_num*"chain_1.csv", DataFrame);

chain_wlwl_ana = CSV.read(path_dzs_wlwl_ana*"chain_1.csv", DataFrame);
chain_gcgc_ana = CSV.read(path_dzs_gcgc_ana*"chain_1.csv", DataFrame);
chain_3x2_ana = CSV.read(path_dzs_3x2_ana*"chain_1.csv", DataFrame);

In [14]:
function mdata(chain, model)
    n = size(chain, 1)
    x = zeros(n, 5)
    xi2 = zeros(n)
    theory = Vector[]
    for i in 1:n
        x = chain[i, 3:7]
        theory_i, xi2_i = model(x)
        xi2[i] = xi2_i
        push!(theory, theory_i)
    end
    return xi2, theory
end

mdata (generic function with 1 method)

In [15]:
xs_wlwl, ts_wlwl = mdata(chain_wlwl_nomarg, model_wlwl)

([245.39635571034546, 243.7762669834875, 243.93941637904823, 243.45460047424203, 243.87550949517694, 243.14478906822367, 242.94027282545957, 244.7603878033697, 249.03674361360248, 243.32403022063636  …  244.41904613128747, 245.79837572356126, 244.25142785900164, 243.1955958678968, 244.4174716806443, 247.71732868470957, 248.29216424596228, 245.44422882728443, 245.88924453122857, 249.05399983519465], Vector[[1.9376823674058986e-9, 1.0141858674626433e-9, 6.602047127216438e-10, 4.842371703982712e-10, 3.8153998910776025e-10, 3.1491249175237404e-10, 2.6821615270674387e-10, 2.326101035170454e-10, 2.0252550869226868e-10, 1.7617093266057847e-10  …  5.438774195778048e-10, 4.709750778786655e-10, 4.0795870307540557e-10, 3.5313510978225847e-10, 3.0526377511309493e-10, 2.6308247822271304e-10, 2.2592039048688268e-10, 1.931480191184344e-10, 1.6426050545081598e-10, 1.3889715326277846e-10], [2.099338618042231e-9, 1.0647190669068274e-9, 6.805088647330855e-10, 4.933075121440897e-10, 3.8501669832043005e-10

In [16]:
CSV.write(path_dzs_wlwl_nomarg*"xi2s_1.csv",
    Dict{String}{Vector}("Xi2s" => xs_wlwl))

CSV.write(path_dzs_wlwl_nomarg*"theory_1.csv",
    Dict{String}{Vector}("Theories" => ts_wlwl))

"chains/nomarg/DES_wlwl_nomarg_TAP_0.65_init_ϵ_0.03/theory_1.csv"

In [17]:
xs_gcgc, ts_gcgc = mdata(chain_gcgc_nomarg, model_gcgc)

([69.84754159466623, 67.43897649629301, 66.71867112222601, 66.97305938460438, 66.53075621249478, 65.05224032654601, 65.80503326808314, 67.32189908910489, 67.73934225495003, 65.8322727144366  …  80.49342799264693, 71.58548059291938, 63.154204712876805, 62.82025724379034, 64.87744559234916, 65.01484005528988, 68.7208904218644, 69.7891792622745, 66.30134743574996, 70.58886236470457], Vector[[0.00015219304333565894, 7.169136149733886e-5, 3.897990109444345e-5, 2.4983414875154113e-5, 1.731830282646847e-5, 8.189246541155057e-5, 5.262907345073108e-5, 3.5526650659723046e-5, 2.3167872373418204e-5, 1.6845762303757436e-5  …  1.7934332739331454e-5, 1.4991849925185267e-5, 1.202580745148194e-5, 9.333111604485904e-6, 7.567673967426092e-6, 6.773528285837296e-6, 5.568858699385652e-6, 4.430909096442361e-6, 3.828205043944368e-6, 3.0342496283242727e-6], [0.00017851119818837629, 7.741949115411632e-5, 3.9739395583518125e-5, 2.4831904250091004e-5, 1.7189476685891488e-5, 9.997744225274618e-5, 5.792396192536445

In [18]:
CSV.write(path_dzs_gcgc_nomarg*"xi2s_1.csv",
    Dict{String}{Vector}("Xi2s" => xs_gcgc))

CSV.write(path_dzs_gcgc_nomarg*"theory_1.csv",
    Dict{String}{Vector}("Theories" => ts_gcgc))


"chains/nomarg/DES_gcgc_nomarg_TAP_0.65_init_ϵ_0.03/theory_1.csv"

In [19]:
xs_3x2, ts_3x2 = mdata(chain_3x2_nomarg, model_3x2)

([509.82906144698416, 516.6852697922396, 514.9397428311933, 514.3758319139521, 511.23140078769984, 513.0911791389261, 510.43187491401005, 511.60397964627634, 508.0344764473468, 508.60399976884986  …  510.43345848310184, 510.2237092925274, 510.458208968243, 514.3149516175198, 515.705119124797, 513.6676781388644, 513.3481401979973, 512.6966528309928, 514.3194427312428, 513.0044500691964], Vector[[0.00016801699759313744, 7.604733263796462e-5, 3.9358495820166816e-5, 2.486240255227723e-5, 1.729603300322288e-5, 9.186769733509949e-5, 5.695541367163226e-5, 3.732064097032292e-5, 2.342496670163876e-5, 1.6935146986468918e-5  …  5.491239634592258e-10, 4.743210713086279e-10, 4.098309389582709e-10, 3.5391806271825745e-10, 3.0521865735762983e-10, 2.6247137417289337e-10, 2.2491537269853412e-10, 1.919048228691866e-10, 1.6290652335044132e-10, 1.3751577157332873e-10], [0.00017553356548093756, 7.650183419530348e-5, 3.927552732680442e-5, 2.442562244774577e-5, 1.6790923644395773e-5, 9.875089054546043e-5, 5.

In [20]:
CSV.write(path_dzs_3x2_nomarg*"xi2s_1.csv",
    Dict{String}{Vector}("Xi2s" => xs_3x2))

CSV.write(path_dzs_3x2_nomarg*"theory_1.csv",
    Dict{String}{Vector}("Theories" => ts_3x2))

"chains/nomarg/DES_3x2_nomarg_TAP_0.65_init_ϵ_0.03/theory_1.csv"

In [ ]:
using Plots

In [ ]:
t, xi2 = model_gcgc(Vector(chain_gcgc_nomarg[1, 3:7]))
println(xi2)

In [ ]:
Vector(chain_gcgc_nomarg[1, 3:7])

In [ ]:
plot(meta_gcgc.data, yscale=:log10, label="Data")
plot!(t, yscale=:log10, label="Model")


In [ ]:
data = log10.(abs.(Matrix(meta_gcgc.inv_cov)))
heatmap(1:size(data,1), 1:size(data,2), data)

In [ ]:
d_gcgc=ones(47)
plot(meta_gcgc.inv_cov*d_gcgc)

In [ ]:
data = log10.(abs.(Matrix(meta_wlwl.inv_cov)))
heatmap(1:size(data,1), 1:size(data,2), data)

In [ ]:
d_wlwl=ones(240)
plot(meta_wlwl.inv_cov*d_wlwl)

In [ ]:
(d_wlwl'*meta_wlwl.inv_cov*d_wlwl)/240

In [ ]:
plot(Matrix(chain_wlwl_nomarg[:, 3:7]))

In [ ]:
plot(Matrix(chain_wlwl_num[:, 3:7]))

In [ ]:
plot(Matrix(chain_gcgc_nomarg[:, 3:3]))

In [ ]:
plot(Matrix(chain_gcgc_nomarg[:, 4:4]))

In [ ]:
plot(Matrix(chain_gcgc_num[:, 3:7]))

In [ ]:
plot(Matrix(chain_gcgc_ana[:, 3:7]))

In [ ]:
plot(Matrix(chain_3x2_nomarg[:, 3:7]))

In [ ]:
histogram(xs_3x2, label="3x2")	

In [ ]:
histogram(xs_gcgc, label="gcgc")

In [ ]:
histogram(xs_wlwl, label="wlwl")